# Phase 0: ML Environment Setup
**DNA Gene Mapping Project - ML Phase**  
**Author:** Sharique Mohammad  
**Date:** February 2026  
**Estimated Time:** 1 hour

## Objective
Verify local environment, establish PostgreSQL connections, validate data, and set up project structure.

## Expected Outcomes
- All 11 tables accessible from PostgreSQL
- Helper functions created for data loading
- Project directories created
- Environment validated and documented

## 1. Environment Verification (15 min)

In [ ]:
# Import core libraries
import sys
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
from sqlalchemy import create_engine, text
import os
from pathlib import Path
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ENVIRONMENT VERIFICATION")
print("="*80)
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")
print(f"SQLAlchemy version: {create_engine.__module__}")

In [ ]:
# Check ML libraries
try:
    import sklearn
    print(f"scikit-learn: {sklearn.__version__}")
except ImportError:
    print("WARNING: scikit-learn not installed")

try:
    import xgboost as xgb
    print(f"XGBoost: {xgb.__version__}")
except ImportError:
    print("WARNING: xgboost not installed")

try:
    import lightgbm as lgb
    print(f"LightGBM: {lgb.__version__}")
except ImportError:
    print("WARNING: lightgbm not installed")

print("\nEnvironment check: PASSED")

## 2. PostgreSQL Connection Test (15 min)

In [ ]:
# Load credentials
load_dotenv()

POSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DB = os.getenv('POSTGRES_DB', 'genome_db')
POSTGRES_USER = os.getenv('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

if not POSTGRES_PASSWORD:
    print("ERROR: PostgreSQL password not found in .env file")
else:
    print("PostgreSQL credentials loaded successfully")

In [ ]:
# Create database connection
conn_str = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(conn_str)

# Test connection
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print("PostgreSQL connection: SUCCESS")
except Exception as e:
    print(f"PostgreSQL connection: FAILED - {e}")

In [ ]:
# Verify 11 tables exist in gold schema
expected_tables = [
    'clinical_ml_features',
    'disease_ml_features',
    'pharmacogene_ml_features',
    'structural_variant_ml_features',
    'variant_impact_ml_features',
    'ml_dataset_variants_train',
    'ml_dataset_variants_validation',
    'ml_dataset_variants_test',
    'ml_dataset_structural_variants_train',
    'ml_dataset_structural_variants_validation',
    'ml_dataset_structural_variants_test'
]

print("\n" + "="*80)
print("TABLE VERIFICATION")
print("="*80)

missing_tables = []
table_info = {}

for table_name in expected_tables:
    try:
        query = f"SELECT COUNT(*) as count FROM gold.{table_name}"
        result = pd.read_sql(query, engine)
        row_count = result['count'].iloc[0]
        table_info[table_name] = row_count
        print(f" {table_name:<50} {row_count:>12,} rows")
    except Exception as e:
        missing_tables.append(table_name)
        print(f" {table_name:<50} MISSING")

if missing_tables:
    print(f"\nERROR: {len(missing_tables)} tables missing")
else:
    print(f"\n SUCCESS: All {len(expected_tables)} tables found")

## 3. Data Loading Utilities (15 min)

In [ ]:
# Helper functions for data loading

def load_table_from_postgres(table_name, limit=None, sample_frac=None):
    """
    Load table from PostgreSQL gold schema
    
    Args:
        table_name: Name of table in gold schema
        limit: Optional row limit
        sample_frac: Optional sampling fraction (0-1)
    
    Returns:
        pandas DataFrame
    """
    query = f"SELECT * FROM gold.{table_name}"
    
    if limit:
        query += f" LIMIT {limit}"
    
    df = pd.read_sql(query, engine)
    
    if sample_frac and 0 < sample_frac < 1:
        df = df.sample(frac=sample_frac, random_state=42)
    
    return df

def get_table_schema(table_name):
    """
    Get column names and types for a table
    """
    query = f"""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_schema = 'gold' AND table_name = '{table_name}'
    ORDER BY ordinal_position
    """
    return pd.read_sql(query, engine)

def get_table_stats(table_name):
    """
    Get basic statistics for a table
    """
    count_query = f"SELECT COUNT(*) as row_count FROM gold.{table_name}"
    count_df = pd.read_sql(count_query, engine)
    
    schema_df = get_table_schema(table_name)
    
    return {
        'row_count': count_df['row_count'].iloc[0],
        'column_count': len(schema_df),
        'columns': schema_df['column_name'].tolist()
    }

print("Helper functions created successfully")

In [ ]:
# Test helper functions with sample data
print("Testing data loading...\n")

# Load small sample
sample_df = load_table_from_postgres('clinical_ml_features', limit=1000)
print(f"Loaded sample: {len(sample_df):,} rows, {len(sample_df.columns)} columns")
print(f"Memory usage: {sample_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Show first few rows
print("\nFirst 3 rows:")
display(sample_df.head(3))

# Test schema function
schema = get_table_schema('clinical_ml_features')
print(f"\nSchema: {len(schema)} columns")
print(schema.head(10))

## 4. Project Structure Setup (15 min)

In [ ]:
# Create output directories
PROJECT_ROOT = Path().absolute().parent

output_dirs = [
    PROJECT_ROOT / 'data' / 'analytical' / 'figures',
    PROJECT_ROOT / 'data' / 'analytical' / 'reports',
    PROJECT_ROOT / 'data' / 'ml' / 'metrics',
    PROJECT_ROOT / 'models'
]

for dir_path in output_dirs:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f" {dir_path}")

print("\nProject structure created successfully")

In [ ]:
# Set random seeds for reproducibility
import random

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# For sklearn (when imported)
try:
    from sklearn.utils import check_random_state
    check_random_state(RANDOM_SEED)
    print(f"Random seed set: {RANDOM_SEED}")
except:
    print(f"NumPy random seed set: {RANDOM_SEED}")

print("Reproducibility configured")

## 5. Environment Report

In [ ]:
# Generate environment status report
report_path = PROJECT_ROOT / 'data' / 'analytical' / 'reports' / 'environment_status.txt'

with open(report_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("ML ENVIRONMENT STATUS REPORT\n")
    f.write("="*80 + "\n\n")
    
    f.write("Python Environment:\n")
    f.write(f"  Python version: {sys.version}\n")
    f.write(f"  Pandas: {pd.__version__}\n")
    f.write(f"  NumPy: {np.__version__}\n\n")
    
    f.write("PostgreSQL Connection:\n")
    f.write(f"  Host: {POSTGRES_HOST}\n")
    f.write(f"  Database: {POSTGRES_DB}\n")
    f.write(f"  Status: Connected\n\n")
    
    f.write("Tables Verified (11 total):\n")
    for table_name, row_count in table_info.items():
        f.write(f"  {table_name}: {row_count:,} rows\n")
    
    f.write("\nProject Directories:\n")
    for dir_path in output_dirs:
        f.write(f"  {dir_path}\n")
    
    f.write("\nRandom Seed: 42\n")
    f.write("\nStatus: READY FOR ML PHASE\n")

print(f"Environment report saved: {report_path}")
print("\n" + "="*80)
print("PHASE 0 COMPLETE")
print("="*80)
print("\nNext: Phase 1 - Exploratory Data Analysis")
print("Open: ml_phase/01_eda/01_disease_ml_features_eda.ipynb")